In [ ]:
import pandas as pd
import altair as alt
import numpy as np

pd.set_option('display.max_columns', None)
# alt.data_transformers.disable_max_rows()

In [ ]:
df = pd.read_csv("../data/raw/SpeedDating.csv",delimiter=",")

In [ ]:
df.head()

Interesting fields:

- gender (Female = 0, Male = 1)
- condtn (Limited choice = 1, extensive choice = 2)
- wave (1-21), especially because of variations
- round (number of people in wave, if not always 10)
- order (number of date that night)
- match (yes = 1, no = 0)
- samerace (yes = 1, no = 0)
- age_o (age of partner)
- race_o (race of partner)
- dec_o (decision of partner the night of event)
- attr_o (rating by partner the night of the event, for all 6 attributes)

Field at time 1 (signup time):
- age
- field (of study)
- undergrd (bachelor student)
- mn_sat (proxy for intelligence)
- race (Black/African American=1, European/Caucasian-American=2, Latino/Hispanic American=3, Asian/Pacific Islander/Asian-American=4, Native American=5, Other=6)
- imprace (importance of same racial/ethic background)
- imprelig (importance of same religion)
- from + zipcode (originally from)
- income (Median household income based on zip code)
- goal (Seemed like a fun night out=1, To meet new people=2, To get a date=3, Looking for a serious relationship=4, To say I did it=5, Other=6)
- date (Several times a week=1, Twice a week=2, Once a week=3, Twice a month=4, Once a month=5, Several times a year=6, Almost never=7)
- go out (Several times a week=1, Twice a week=2, Once a week=3, Twice a month=4, Once a month=5, Several times a year=6, Almost never=7)
- career
- interest in activities from 1-10
- exphappy from 1-10 (how happy do you expect to be with prople you meet during the speed-dating event)
- expnum (out of 20 people, how many do you expect to be interested in you?)

Fields at time 2 (follow up):
- satis_2 (how satisfied, 1 - 10)

Field at time 3 (followup2, 3-4 after they knew their matches):
- you_call (how many have you contacted)
- them_cal (how many contacted you)
- date_3 (have you been on a date? yes = 1, no = 2)
- numdat_3 (how many dates?)
- num_in_3


In [ ]:
# alt.Chart(df).mark_bar().encode(
#     alt.X("you_call:O"),
#     alt.Y("count(you_call)")
# ) | alt.Chart(df).mark_bar().encode(
#     alt.X("them_cal:O"),
#     alt.Y("count(them_cal)")
# )


In [ ]:
# c1 = alt.Chart(df).mark_bar().encode(
#     alt.X("samerace:O"),
#     alt.Y("mean(imprace)")
# )

# c2 = alt.Chart(df).mark_bar().encode(
#     alt.X("samerace:O"),
#     alt.Y("mean(imprelig)")
# )

# c1 | c2

In [ ]:
# alt.Chart(df).mark_bar().encode(
#     alt.X("age_o"),
#     alt.Y("count(match)"),
#     alt.Color("match:O")
# )

In [ ]:
# alt.Chart(df).mark_bar().encode(
#     alt.X("condtn:O", axis = alt.Axis(title="Conditional")),
#     alt.Y("count(match)"),
#     alt.Color("match:O")
# )